# Градиентный бустинг

Бустинг это метод построения компизиции алгоритмов, в котором базовые алгоритмы строятся последовательно один за другим, причем каждый следующий алгоритм строится таким образом, чтобы уменьшить ошибку предыдущего.

Положим, что алгоритм это линейная комбинация некоторых базовых алгоритмов:
    $$a_N(x) = \sum_{n=1}^N b_n(x)$$

Пусть задана некоторая функция потетерь, которую мы оптимизируем
$$\sum_{i=1}^l L(\hat y_i, y_i) \rightarrow min$$ 


Зададимся вопросом: а что если мы хотим добавить ещё один алгоритм в эту композицию, но не просто добавить, а как можно оптимальнее с точки зрения исходной оптимизационной задачи. То есть уже есть какой-то алгоритм $a_N(x)$ и мы хотим прибавить к нему базовый алгоритм $b_{N+1}(x)$:

$$\sum_{i=1}^l L(a_{N}(x_i) + b_{N+1}(x_i), y_i) \to \min_{b_{N+1}}$$

Сначала имеет смысл решить более простую задачу: определить, какие значения $r_1 ,r_2 ..., r_l$ должен принимать алгоритм $b_N(x_i) = r_i$ на объектах обучающей выборки, чтобы ошибка на обучающей выборке была минимальной:

$$F(r) = \sum_{i=1}^l L(a_{N}(x_i) + r_i, y_i) \to \min_{r},$$

где $r = (r_1, r_2, \dots, r_l)$ - вектор сдвигов.

Поскольку направление наискорейшего убывания функции задается направлением антиградиента, его можно принять в качестве вектора $r$:
$$r = -\nabla F \\$$
$$r_i = \frac{\partial{L}(a_N(x_i), y_i))}{\partial{a_N(x_i)}}, \ \ \ i = \overline{1,l}$$

Компоненты вектора $r$, фактически, являются теми значениями, которые на объектах обучающей выборки должен принимать новый алгоритм $b_{N+1}(x)$, чтобы минимизировать ошибку строящейся композиции. 
Обучение $b_{N+1}(x)$, таким образом, представляет собой *задачу обучения на размеченных данных*, в которой ${(x_i , r_i )}_{i=1}^l$ — обучающая выборка, и используется, например, квадратичная функция ошибки:
$$b_{N+1}(x) = arg \min_{b}\sum_{i=1}^l(b(x_i) - r_i)^2$$

Таким образом, можно подобрать неплохое улучшение текущего алгоритма $a_N(x)$, а потом ещё раз и ещё, в итоге получив комбинацию алгоритмов, которая будет минимизировать исходный функционал.

# Бустинг над решающими деревьями

Наиболее популярное семейство алгоритмов для бустинга это деревья. Рассмотрим популярные библиотеки

In [2]:
import time 
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, roc_auc_score
roc_auc_scorer = make_scorer(roc_auc_score)

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

/Users/vladvo/envs/py3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


## Данные

### amazon

https://www.kaggle.com/c/amazon-employee-access-challenge/data

In [12]:
amazon_data = pd.read_csv("data/amazon.csv")

In [13]:
amazon_data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [14]:
X, y = amazon_data.drop('ACTION', axis=1).values, amazon_data['ACTION'].values

### hr

In [6]:
data = pd.read_csv('data/HR.csv')

In [7]:
data.head()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.53,2,157,3,0,1,0
1,0.86,5,262,6,0,0,0
2,0.88,7,272,4,0,1,0
3,0.87,5,223,5,0,1,0
4,0.52,2,159,3,0,1,0


In [8]:
X, y = data.drop('left', axis=1).values, data['left'].values

## ROC AUC по 100 деревьев:

In [15]:
for name, factory in [
    ("xgboost", lambda: XGBClassifier(n_estimators=100)),
    ("lightgbm", lambda: LGBMClassifier(n_estimators=100, learning_rate=0.1)),
    ("catboost", lambda: CatBoostClassifier(verbose=False, iterations=100, learning_rate=0.1))
]:
    start = time.time()
    scores = cross_val_score(factory(), X, y, scoring=roc_auc_scorer)
    mean_score = scores.mean()
    total_time = (time.time() - start) * 1000.
    print(f'{name} - {mean_score:.3f} took {total_time:.1f} ms')

xgboost - 0.502 took 4699.8 ms
lightgbm - 0.557 took 820.4 ms
catboost - 0.506 took 11216.7 ms


## Больше деревьев, лучше качество

In [22]:
for n in [64, 128, 256, 512, 1024, 2048]:
    start = time.time()
    scores = cross_val_score(LGBMClassifier(n_estimators=n, learning_rate=0.3§), X, y, scoring=roc_auc_scorer)
    mean_score = scores.mean()
    total_time = (time.time() - start) * 1000.
    print(f'lightgbm_{n} - {mean_score:.3f} took {total_time:.1f} ms')

lightgbm_64 - 0.598 took 498.1 ms
lightgbm_128 - 0.618 took 1072.4 ms
lightgbm_256 - 0.636 took 1807.0 ms
lightgbm_512 - 0.642 took 3173.5 ms
lightgbm_1024 - 0.644 took 6021.7 ms
lightgbm_2048 - 0.647 took 14859.1 ms


## Увеличим число деревьев и уменьшим Learning rate

In [23]:
for n in [2048, 4096, 8192]:
    start = time.time()
    scores = cross_val_score(LGBMClassifier(n_estimators=n, learning_rate=0.3 * 2048 / n), X, y, scoring=roc_auc_scorer)
    mean_score = scores.mean()
    total_time = (time.time() - start) * 1000.
    print(f'lightgbm_{n} - {mean_score:.3f} took {total_time:.1f} ms')

lightgbm_2048 - 0.647 took 14723.0 ms
lightgbm_4096 - 0.645 took 25445.4 ms
lightgbm_8192 - 0.647 took 50680.9 ms


## Можно применять не все деревья

Например если делаем сервис для прода и нагрузка возрастет в 5 раз